# Classifying with probability theory: naïve Bayes

**Bayes' Rule**

Bayes' rule tells us how to swap the symbols in a conditional probability statement. If we have P(B|A) but want to have P(A|B), we can find it with the following:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

**Bayes's Decision**

Assume for a moment that we have a dataset with two classes of data inside. For data x, we have probabilities: \\(p(c_1|x)\\) and \\(p(c_2|x)\\).
* If \\(p(c_1|x) > p(c_2|x)\\), then the class is c1.
* If \\(p(c_1|x) < p(c_2|x)\\), then the class is c2.

**General approach to naïve Bayes**
1. Collect: Any method. We’ll use RSS feeds in this chapter.
2. Prepare: Numeric or Boolean values are needed.
3. Analyze: With many features, plotting features isn’t helpful. Looking at histograms is a better idea.
4. Train: Calculate the conditional probabilities of the independent features.
5. Test: Calculate the error rate.
6. Use: One common application of naïve Bayes is document classification. You can use naïve Bayes in any classification setting. It doesn’t have to be text.

In [ ]:
import numpy as np

## Classifying text
Let's apply Bayes' rule to classify text. \\(p(c_i|w)\\) means the probability that the word \\(w\\) belongs to the class \\(c_i\\). How to calculate \\(p(c_i|w)\\)?
$$p(c_i|w) = \frac{p(w|c_i)p(c_i)}{p(w)}$$

You know whether a word occurs in a document, and you know what class the document belongs to. So, you can calculate \\(p(w|c_i)\\) and \\(p(c_i)\\).

In [ ]:
# data for test
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how','to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

# create a vocabulary
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

# convert a document into a vector on the vocabulary
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my vocabulary!" % word)
    return returnVec

In [ ]:
listPosts, listClasses = loadDataSet()

In [ ]:
myVocabList = createVocabList(listPosts)
myVocabList

In [ ]:
setOfWords2Vec(myVocabList, listPosts[0])

In [ ]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = np.sum(trainCategory) / float(numTrainDocs)
    # p0Num, p1Num = np.zeros(numWords), np.zeros(numWords)
    # p0Denom, p1Denom = 0.0, 0.0
    p0Num, p1Num = np.ones(numWords), np.ones(numWords)
    p0Denom, p1Denom = 2.0, 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += np.sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += np.sum(trainMatrix[i])
    # p1Vec = p1Num / p1Denom
    # p0Vec = p0Num / p0Denom
    p1Vec = np.log(p1Num / p1Denom)
    p0Vec = np.log(p0Num / p0Denom)
    return p0Vec, p1Vec, pAbusive

In [ ]:
trainMat = []
for postDoc in listPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postDoc))

In [ ]:
len(trainMat), len(trainMat[0]), np.sum(listClasses)

In [ ]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [ ]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [ ]:
testDoc = np.array(setOfWords2Vec(myVocabList, ['love', 'my', 'damation']))
classifyNB(testDoc, p0V, p1V, pAb)

In [ ]:
testDoc = np.array(setOfWords2Vec(myVocabList, ['stupid', 'garbage']))
classifyNB(testDoc, p0V, p1V, pAb)

In [ ]:
def testingNB():
    listPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listPosts)
    trainMat = []
    for postinDoc in listPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
        p0V,p1V,pAb = trainNB0(np.array(trainMat), np.array(listClasses))
        testEntry = ['love', 'my', 'dalmation']
        thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
        print(testEntry, 'classified as: ', classifyNB(thisDoc,p0V,p1V,pAb))
        testEntry = ['stupid', 'garbage']
        thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
        print(testEntry,'classified as: ', classifyNB(thisDoc,p0V,p1V,pAb))

In [ ]:
testingNB()

In [ ]:
def bagOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print("the word: %s is not in my vocabulary!" % word)
    return returnVec

## Classifying spam email

In [ ]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent.split()

In [ ]:
import re

In [ ]:
regEx = re.compile('\\W*')
listOfTokens = regEx.split(mySent)

In [ ]:
listOfTokens

In [ ]:
[tok for tok in listOfTokens if len(tok) > 0]

In [ ]:
[tok.lower() for tok in listOfTokens if len(tok) > 0]

In [ ]:
import re
def textParse(bigString):
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [ ]:
def spamTest():
    docList, classList, fullText = [], [], []
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = [i for i in range(50)]; testSet = []
    for i in range(10):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat, trainClasses = [], []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount/len(testSet)))    

In [ ]:
spamTest()

## Reveal local attitudes from personal ads

In [ ]:
import feedparser

In [ ]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

In [ ]:
ny